# Spatiocyte simulations with single-molecule resolution

We showed an example of E-Cell4 spatial representation.  
Next let's simulate the models with more detailed spatial representation called **single molecule resolution**.

## Spatiocyte lattice-based method

In spatical Gillespie method, we divided the **Space** into smaller **Space**, then we diffuse the molecules in the **Subvolume**.
However we simulated the molecules in the **Subvolume** as the number of the molecules, and the location of the molecules are NOT determinated.

In other words, the spatical resolution of spatical Gillespie method is equal to the side of the **Subvolume** $l$.
To improve this resolution we need to make the size of $l$ small, but in this method the $l$ must be larger than the (at least) 10 times the diameter of molecule $R$.

How can we improve the spatical resolution to the size of the molecule?
The answer is the simulation with single-molecule resolution.
This method simulate the molecule not with the number of the molecules, but with the spatical reaction diffusion of each molecule.

E-Cell4 has multiple single-molecule resolution method, here we explain about Spatiocyte lattice-based method.

Spatiocyte treat each molecule as hard sphere and diffuse the molecules in hexagonal close-packed lattice.

Spatiocyte has ID for each molecule and the position of the molecule with single-molecule resolution.
To use the time scale, Spatiocyte has 100 times smaller time-step than spatical Gillespie, because the time scale of diffusion increases with the square of the distance.

Next, let's use Spatiocyte.


In [1]:
from ecell4 import *

with species_attributes():
    A | B | C | {'D': '1'}

with reaction_rules():
    A + B == C | (0.01, 0.3)

m = get_model()
w = lattice.LatticeWorld(Real3(1, 1, 1), 0.005)  # The second argument is 'voxel_radius'.
w.bind_to(m)
w.add_molecules(Species('C'), 60)
sim = lattice.LatticeSimulator(w)
obs = FixedIntervalNumberObserver(0.1, ('A', 'B', 'C'))
sim.run(10, obs)

There is a distinct difference in the second argument for **LatticeWorld**. This is called **Voxel radius**.
Spatiocyte defines the locations of the molecules with dividing the Space with molecule size, and call the minimum unit for this Space as **Voxel**.

In most cases the size of the molecule would be good for **Voxel radius**.

In this example, we set 5 $\mathrm{nm}$ as the radius of the molecule in the Space with the side 1 $\mathrm{\mu m}$ .

It takes more time to simulate, but the result is same with ODE or Gillespie.

## The diffusion movement of single molecule

Next let's simulate single molecule diffusion to check the resolution.


In [2]:
from ecell4 import *

with species_attributes():
    A | {'D': '1'}

m = get_model()
w = lattice.LatticeWorld(Real3(1, 1, 1), 0.005)
w.bind_to(m)

(pid, p), suc = w.new_particle(Species('A'), Real3(0.5, 0.5, 0.5))

**new_particle** method places a particle to a coordinate in **LatticeWorld**, and returns the particle's **pid**, the information about the particle **p**, and verify whether the particle is cooked with **suc**.
If a particle is already placed in the coordinate you can NOT place a particle over it and **suc** will be False and fail.

**p** contains the particle position, species type, radius, and diffusion coefficient.
You can inspect the **p** with the particle's ID **pid**.

Let's check **p**.

In [3]:
pid, p = w.get_particle(pid)
print(p.species().serial())  # will print: A
print(p.radius(), p.D())  # will print: (0.005, 1.0)
print(tuple(p.position()))  # will print: (0.49806291436591293, 0.49652123150307814, 0.5)

A
0.005 1.0
(0.49806291436591293, 0.49652123150307814, 0.5)


**get_particle** method receives the particle ID and returns the ID and particle (of cource the IDs are same).
You can inspect the coordinate of the particle as **Real3** with **position** method.
It is hard to directly read the coordinate, here we printed it after converting to tuple.
As you can see the tuple coodinate is slightly different from original **Real3**. This is because Spatiocyte can place the molecule only on the lattice.
**LatticeWorld** places the molecule the nearest lattice for the argument **Real3**.

You can visualize the coordinate of the molecule with **viz.plot_world** method, and check the molecule in the center of the World.


In [4]:
viz.plot_world(w, save_image=True)

{'A': '#a6cee3'}

And you can use **Observer** to track the trajectory of molecular diffusion process.

In [5]:
sim = lattice.LatticeSimulator(w)
obs = FixedIntervalTrajectoryObserver(0.002, (pid,))
sim.run(1, obs)
viz.plot_trajectory(obs)

{'1': '#a6cee3'}

Here we visualized the trajectory with **viz.plot_trajectory** method, you can also obtain it as Real3 list with **data** method.


In [8]:
print(len(obs.data()))
print(len(obs.data()[0]))

1
501


**data** method returns nested list.
First index means the index of the particle.
Second index means the index of the **Real3**.
In this case we threw just one particle, so the first result is **1**, and next **501** means time-series coordinate of the only one particle (initial coordinate and the coordinates in 1/0.002 = 500 time points).

Also you can obtain the particles in bulk with **list_particles** method and species type.

In [7]:
w.add_molecules(Species('A'), 5)

particles = w.list_particles(Species('A'))
for pid, p in particles:
    print(p.species().serial(), tuple(p.position()))

A (0.6940220937885672, 0.42723919920032305, 0.79)
A (0.12247448713915891, 0.8256108849411649, 0.96)
A (0.47356801693808115, 0.9699484522385713, 0.11)
A (0.563382640840131, 0.3925981830489455, 0.13)
A (0.23678400846904057, 0.5051814855409226, 0.195)
A (0.07348469228349536, 0.002886751345948129, 0.8250000000000001)


Please remember **list_particles** method, this method can be used for other World as well as **add_molecules** method.

On a different note, in Spatiocyte proper method to inspect the single molecule is **list_voxels**, and the coordinate is described with index of voxel (not Real3).

## The diffusion coefficient and the second-order reaction

The models we have addressed are called **second-order reaction**.
Let's look at the relationship between this second-order reaction and the diffusion coefficient in Spatiocyte.

In [8]:
from ecell4 import *

with species_attributes():
    A | B | C | {'D': '1'}

with reaction_rules():
    A + B > C | 1.0

m = get_model()
w = lattice.LatticeWorld(Real3(2, 1, 1), 0.005)
w.bind_to(m)
w.add_molecules(Species('A'), 120)
w.add_molecules(Species('B'), 120)
obs = FixedIntervalNumberObserver(0.005, ('A', 'B', 'C'))
sim = lattice.LatticeSimulator(w)
sim.run(1.0, obs)

## The structure of Spatiocyte method

## The structure and the reaction